In [1]:
#Sprawdzenie dostępności GPU
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 1


In [2]:
#Pobranie danych HIGGS (~2.5 GB)
!wget -q https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz
!gunzip -f HIGGS.csv.gz

In [4]:
#Pipeline danych
CSV_FILE = "HIGGS.csv"
FEATURE_DIM = 28
BATCH_SIZE = 1024


In [5]:
def parse_csv(line):
    defaults = [[0.0]] * (FEATURE_DIM + 1)
    fields = tf.io.decode_csv(line, record_defaults=defaults)
    label = fields[0]
    features = tf.stack(fields[1:])
    return features, label

In [6]:
dataset = (
    tf.data.TextLineDataset(CSV_FILE)
    .map(parse_csv, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(100_000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)


In [7]:
train_ds = dataset.take(1000)  # ≈ 1 mln przykładów


In [8]:
#Model na GPU
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(FEATURE_DIM,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [9]:
#Trening (na GPU)
model.fit(train_ds, epochs=5)

#Ewaluacja (opcjonalnie)
eval_ds = dataset.skip(1000).take(200)
model.evaluate(eval_ds)

Epoch 1/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 92s 84ms/step - accuracy: 0.6269 - loss: 0.6402
Epoch 2/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 89s 81ms/step - accuracy: 0.6861 - loss: 0.5881
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 88s 80ms/step - accuracy: 0.6989 - loss: 0.5723
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 143s 82ms/step - accuracy: 0.7085 - loss: 0.5606
Epoch 5/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 140s 83ms/step - accuracy: 0.7164 - loss: 0.5513
200/200 ━━━━━━━━━━━━━━━━━━━━ 99s 81ms/step - accuracy: 0.7228 - loss: 0.5414


[0.5417237281799316, 0.7232568264007568]